In [12]:
import pickle
import re
from tqdm import tqdm
import time
import numpy as np
from IPython.display import clear_output 
import pandas as pd
import time

In [4]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [5]:
wait_time = 0.5

In [7]:
# chromedriver setup

serv = Service(executable_path=r'C:\Users\SamueleColombo\OneDrive - S2E Sprint srl\Documenti\bbway\chromedriver\chromedriver.exe') #path from 'which chromedriver'

# test driver
# for headless chrome mode
chrome_options = Options()

In [34]:
# assemble list of links to all articles
links = []

# interact with cookie terms
chrome_path = r'C:\Users\SamueleColombo\OneDrive - S2E Sprint srl\Documenti\bbway\chromedriver' #path from 'which chromedriver'
chrome_options = Options()
#chrome_options.add_argument("--headless") # open chrome in background
driver = webdriver.Chrome(service=serv, options=chrome_options)
driver.get('https://www.allsides.com/headline-roundups')
wait = WebDriverWait(driver, 10)
wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'banner-actions-container')))
ele = driver.find_element(By.CLASS_NAME, "banner-actions-container")
ele.click()
# retireve number of pages 
last_page_button = driver.find_element(By.CLASS_NAME, "pager-last")
link_last_page = last_page_button.find_elements(By.TAG_NAME, "a")
t = link_last_page[0].get_attribute("href")
last_page_index = int(t[-3:])

# retrieve links from start page
main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

rows = main_table.find_elements(By.TAG_NAME, "tr")
for i in rows:
    entry = i.find_elements(By.TAG_NAME, "td")
    link = entry[0].find_element(By.TAG_NAME, "a")
    links.append(link.get_attribute("href"))
WebDriverWait(driver, 20)

# retrieve links for other pages
for page in tqdm(range(2,last_page_index+1)):   # set to max number of pages
    driver.get("https://www.allsides.com/headline-roundups?page="+str(page))
    WebDriverWait(driver, 20)
    main_table = driver.find_element(By.XPATH, "//*[@id=\"block-views-de37fa32ea86f5545eb9b7722977a70d\"]/div/div[2]/table/tbody") # table body

    rows = main_table.find_elements(By.TAG_NAME, "tr")
    for i in rows:
        entry = i.find_elements(By.TAG_NAME, "td")
        link = entry[0].find_element(By.TAG_NAME, "a")
        links.append(link.get_attribute("href"))

driver.close()

100%|██████████| 199/199 [10:31<00:00,  3.18s/it]


In [35]:
# exports links as pickle file
with open("linklist_allsides_news.pickle", "wb") as f:
    pickle.dump(links, f, pickle.HIGHEST_PROTOCOL)

In [36]:
# exports links as csv
with open("linklist_allsides_news.csv", "w") as f:
    for line in links:
        print(line, file=f)

In [8]:
# functions that assert the existence of 
def check_exists_by_xpath(xpath):
    try:
        driver.find_elements(By.XPATH, xpath)[0]
    except:
        return False
    return True

def check_exists_by_class(inp):
    try:
        driver.find_elements(By.CLASS_NAME, inp)[0]
    except:
        return False
    return True

In [9]:
# load link list from pickle file
with open("linklist_allsides_news.pickle", "rb") as f:
    links = pickle.load(f)

links

['https://www.allsides.com/story/americas-anti-nato-pro-palestinian-protests-turn-violent-montreal',
 'https://www.allsides.com/story/criminal-justice-illinois-supreme-court-overturns-conviction-against-jussie-smollett',
 'https://www.allsides.com/story/donald-trump-who-trumps-new-attorney-general-pick-pam-bondi',
 'https://www.allsides.com/story/sexual-misconduct-newly-public-police-report-details-hegseth-assault-allegation',
 'https://www.allsides.com/story/media-industry-comcast-splits-causing-ramifications-cable-news',
 'https://www.allsides.com/story/politics-gaetz-withdraws-attorney-general-nominee',
 'https://www.allsides.com/story/middle-east-icc-issues-arrest-warrants-netanyahu-gallant-hamas-commander',
 'https://www.allsides.com/story/voting-rights-and-voter-fraud-viral-voter-id-fraud-claims-are-countered-left',
 'https://www.allsides.com/story/politics-federal-judge-blocks-biden-admin-s-overtime-pay-rule',
 'https://www.allsides.com/story/politics-house-ethics-committee-dead

In [21]:
# # list for results 
# data = []

# # retrieve information from articles in list of links
# for li in tqdm(links):
#     time.sleep(wait_time)
#     driver = webdriver.Chrome(service=serv, options=chrome_options)
#     print(li)
#     # open URL
#     driver.get(li)
    
#     # interact with pop-up window
#     if check_exists_by_class("ot-sdk-show-settings"):
#         ele = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((driver.find_element(By.CLASS_NAME, "banner-actions-container"))))
#         ele.click()
#     else:
#         print("no coockie button")

    
#     # netral title heading 
#     try:
#         heading = driver.find_element(By.TAG_NAME, "h1").text
#     except:
#         print("no heading found")
    
#     print(heading)

    
#     # date
#     try:
#         date = driver.find_element(By.CLASS_NAME, "date-display-single").text
#     except:
#         date = ""
    
#     # tags
#     try:
#         tags = [a.text for a in driver.find_element(By.CLASS_NAME, "page-tags").find_elements(By.TAG_NAME, "a")]
#     except:
#         tags = ""
    
#     print(tags)
    
#     # define XPATH inforamtion for article divs
#     divs = ["/html/body/div[4]/div/div/div/div[4]/div/div/div/div[1]", "/html/body/div[4]/div/div/div/div[4]/div/div/div/div[2]", "/html/body/div[4]/div/div/div/div[4]/div/div/div/div[3]"]
 
#     # access information in article divs
#     for d in divs:
#         if check_exists_by_xpath(d):
#             div = driver.find_elements(By.XPATH, d)[0]

#             # check heading element to find out left/center/right. The title contains the bias label that we can retrieve from the text here
#             try:
#                 cat = div.find_element(By.TAG_NAME, "h3").text
#             except: 
#                 print("no headline found")
            
#             print("cat ",cat)
                
#             # retrieve link to original article
#             try:
#                 link = div.find_element(By.TAG_NAME, "a").get_attribute("href")
#                 print(link)
#             except:
#                 print("no link found")
            
#             print("link ",link)
                
#             # left/center/right are shuffled for each article, some roundups have e.g. only left and right articles. 
#             # Thus, we have look to look each article seperately
#             time.sleep(0.2)
#             if "Left" in cat:
#                 print("left")
#                 try:
#                     left_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading       
#                 except:
#                     left_heading = ""
#                     print("no headline found")
#                 try:
#                     left_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source
#                 except:
#                     left_source = ""
#                     print("no source found")
#                 try:
#                     left_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text body-contents
#                 except:
#                     left_text = ""
#                     print("no text found")
                    
#                 # add the article information
#                 data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":left_heading, "source": left_source, "text": left_text, "bias_rating": "left"})
#             elif "Right" in cat:
#                 print("right")
#                 try:
#                     right_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
#                 except:
#                     right_heading = ""
#                     print("no headline found")
#                 try:
#                     right_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
#                 except:
#                     right_source = ""
#                     print("no source found")
#                 try:
#                     right_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
#                 except:
#                     right_text = ""
#                     print("no text found")
                
#                 # add the article information
#                 data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":right_heading, "source": right_source, "text": right_text, "bias_rating": "right"}) 

#             else:
#                 print("center")
#                 try:
#                     center_heading = div.find_element(By.CLASS_NAME, "news-title").text # heading
#                 except:
#                     center_heading = ""
#                     print("no headline found")
#                 try:
#                     center_source = div.find_element(By.CLASS_NAME, "source-area").find_element(By.TAG_NAME, "span").text #source 
#                 except:
#                     center_source = ""
#                     print("no source found")
#                 try:
#                     center_text = div.find_element(By.CLASS_NAME, "news-body").find_element(By.CLASS_NAME, "body-contents").text # news text
#                 except:
#                     center_text = ""
#                     print("no text found")
                    
#                 # add the article information
            
#                 data.append({"url":link, "date":date, "title": heading, "tags": tags, "heading":center_heading, "source": center_source, "text": center_text, "bias_rating": "center"})
#         else:
#             print("div not found")
    
#     break
#     # clear output
#     clear_output()
#     # added a wait here to assert the scraper runs well
#     time.sleep(wait_time)

    


  0%|          | 0/9983 [00:00<?, ?it/s]

https://www.allsides.com/story/americas-anti-nato-pro-palestinian-protests-turn-violent-montreal
no coockie button
Anti-NATO, Pro-Palestinian Protests Turn Violent in Montreal


  0%|          | 0/9983 [00:04<?, ?it/s]

['The Americas', 'NATO', 'Palestine', 'Israel Hamas Violence', 'Protests', 'Canada', 'Justin Trudeau']
div not found
div not found
div not found


In [29]:
links[:10]

['https://www.allsides.com/story/americas-anti-nato-pro-palestinian-protests-turn-violent-montreal',
 'https://www.allsides.com/story/criminal-justice-illinois-supreme-court-overturns-conviction-against-jussie-smollett',
 'https://www.allsides.com/story/donald-trump-who-trumps-new-attorney-general-pick-pam-bondi',
 'https://www.allsides.com/story/sexual-misconduct-newly-public-police-report-details-hegseth-assault-allegation',
 'https://www.allsides.com/story/media-industry-comcast-splits-causing-ramifications-cable-news',
 'https://www.allsides.com/story/politics-gaetz-withdraws-attorney-general-nominee',
 'https://www.allsides.com/story/middle-east-icc-issues-arrest-warrants-netanyahu-gallant-hamas-commander',
 'https://www.allsides.com/story/voting-rights-and-voter-fraud-viral-voter-id-fraud-claims-are-countered-left',
 'https://www.allsides.com/story/politics-federal-judge-blocks-biden-admin-s-overtime-pay-rule',
 'https://www.allsides.com/story/politics-house-ethics-committee-dead

In [30]:
# list for results 
data = []

# retrieve information from articles in list of links
for li in tqdm(links[:10]):
    time.sleep(wait_time)
    driver = webdriver.Chrome(service=serv, options=chrome_options)
    print(li)
    # open URL
    driver.get(li)
    
    # interact with pop-up window
    try:
        cookie_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "ot-sdk-btn"))
        )
        cookie_button.click()
    except:
        print("no cookie button or already accepted")

    # neutral title heading 
    try:
        heading = driver.find_element(By.CLASS_NAME, "taxonomy-heading").text
    except:
        print("no heading found")
        heading = ""
    
    print(heading)

    # date
    try:
        date = driver.find_element(By.CLASS_NAME, "date-display-single").text
    except:
        date = ""
    
    # tags
    try:
        tags = [a.text for a in driver.find_element(By.CLASS_NAME, "page-tags").find_elements(By.TAG_NAME, "a")]
    except:
        tags = ""
    
    print(tags)
    
    # Find all news items within featured coverage
    try:
        news_items = driver.find_element(By.CLASS_NAME, "featured-coverage").find_elements(By.CLASS_NAME, "news-item")
        
        for item in news_items:
            # Determine bias based on class
            if "left" in item.get_attribute("class").lower():
                bias = "left"
            elif "center" in item.get_attribute("class").lower():
                bias = "center"
            elif "right" in item.get_attribute("class").lower():
                bias = "right"
            else:
                continue
                
            try:
                item_heading = item.find_element(By.CLASS_NAME, "news-title").text
            except:
                item_heading = ""
                print("no headline found")
                
            try:
                item_source = item.find_element(By.CLASS_NAME, "news-source").text
            except:
                item_source = ""
                print("no source found")
                
            try:
                item_text = item.find_element(By.CLASS_NAME, "body-contents").text
            except:
                item_text = ""
                print("no text found")
                
            try:
                item_link = item.find_element(By.TAG_NAME, "a").get_attribute("href")
            except:
                item_link = ""
                print("no link found")
            
            # add the article information
            data.append({
                "url": item_link,
                "date": date,
                "title": heading,
                "tags": tags,
                "heading": item_heading,
                "source": item_source,
                "text": item_text,
                "bias_rating": bias
            })
            
    except:
        print("No featured coverage found")
    
    driver.quit()
    # clear output
    clear_output()
    # added a wait here to assert the scraper runs well
    time.sleep(wait_time)


100%|██████████| 10/10 [01:39<00:00,  9.90s/it]


In [31]:
## convert data to dataframe
df = pd.DataFrame(data)

# export scraped articles
df.to_csv("allsides_news_complete.csv")

In [32]:
df

,url,date,title,tags,heading,source,text,bias_rating
0,https://www.cbc.ca/news/canada/montreal/anti-n...,"November 24th, 2024","Anti-NATO, Pro-Palestinian Protests Turn Viole...","[The Americas, NATO, Palestine, Israel Hamas V...","Trudeau, political leaders denounce 'violence ...",CBC News,Prime Minister Justin Trudeau has joined feder...,left
1,https://www.theglobeandmail.com/canada/article...,"November 24th, 2024","Anti-NATO, Pro-Palestinian Protests Turn Viole...","[The Americas, NATO, Palestine, Israel Hamas V...",Montreal anti-NATO protest ends in three arres...,The Globe and Mail,Montreal police clashed with anti-NATO and pro...,center
2,https://www.bbc.com/news/articles/c8xp28eq5l9o,"November 22nd, 2024",Illinois Supreme Court Overturns Conviction ag...,"[Criminal Justice, Jussie Smollett, Conviction...",Illinois Supreme Court overturns Jussie Smolle...,BBC News,The Illinois Supreme Court has overturned a co...,center
3,https://www.foxnews.com/entertainment/jussie-s...,"November 22nd, 2024",Illinois Supreme Court Overturns Conviction ag...,"[Criminal Justice, Jussie Smollett, Conviction...",Jussie Smollett conviction overturned by Illin...,Fox News Digital,The Illinois Supreme Court on Thursday overtur...,right
4,https://apnews.com/article/jussie-smollett-con...,"November 22nd, 2024",Illinois Supreme Court Overturns Conviction ag...,"[Criminal Justice, Jussie Smollett, Conviction...",Jussie Smollett’s conviction in 2019 attack on...,Associated Press,The Illinois Supreme Court on Thursday overtur...,left
5,https://www.newsweek.com/what-was-matt-gaetz-a...,"November 22nd, 2024","Who is Trump's New Attorney General Pick, Pam ...","[Donald Trump, Attorney General, Matt Gaetz, P...",What Was the Matt Gaetz Attorney General Pick ...,Josh Hammer,Barely a week after he was announced as Presid...,right
6,https://www.washingtonpost.com/politics/2024/1...,"November 22nd, 2024","Who is Trump's New Attorney General Pick, Pam ...","[Donald Trump, Attorney General, Matt Gaetz, P...","Pam Bondi, Trump’s pick for attorney general, ...",Washington Post,President-elect Donald Trump announced that he...,left
7,https://www.cbsnews.com/news/pete-hegseth-meet...,"November 22nd, 2024",Newly-Public Police Report Details Hegseth Ass...,"[Donald Trump, Pete Hegseth, Trump Administrat...",Pete Hegseth meets with Senate Republicans to ...,CBS News (Online),"Former Fox News host Pete Hegseth, President-e...",left
8,https://www.washingtonexaminer.com/news/323720...,"November 22nd, 2024",Newly-Public Police Report Details Hegseth Ass...,"[Donald Trump, Pete Hegseth, Trump Administrat...",Police report reveals new details about Pete H...,Washington Examiner,A newly released police report from 2017 alleg...,right
9,https://www.wsj.com/politics/police-report-rev...,"November 22nd, 2024",Newly-Public Police Report Details Hegseth Ass...,"[Donald Trump, Pete Hegseth, Trump Administrat...",Trump Team Blindsided by Details of Sexual-Ass...,Wall Street Journal (News),Members of President-elect Donald Trump’s tran...,center
